Dámaso Sánchez Arenas
Eduardo Rodríguez de Castro
Grupo 09

In [28]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report

In [2]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(ngram_range = (1,2), stop_words = ['and',',','.','I','is','that','or','a','of','you','to','for'])
tfidfer= TfidfTransformer()

# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
train_vector_data=vectorizer.fit_transform(train_data.data)
# Calculamos el valor TF-IDF 
train_preprocessed=tfidfer.fit_transform(train_vector_data)

# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
test_vector_data=vectorizer.transform(test_data.data)
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
test_preprocessed=tfidfer.transform(test_vector_data)

En stopwords debemos meter la lista de palabras que queremos quitar del texto. Se utiliza ya que son palabras irrelevantes, es decir, palabras que no dan informacion y que lo unico que hacen es complicar la clasificacion. 
Las palabras que aparecen con frecuencia entre los documentos no son buenas para la recuperación de información.

Se puede quitar cualquier tipo de palabra, Los artículos, los pronombres, las preposiciones, y las conjunciones son candidatos naturales.

Ademas, algunos verbos, adverbios,adjetivos y los términos específicos de un dominio se podían tratar como stopwords.
Eliminar este tipo de palabras vacias mejorarian el analis lexico haciendolo mas eficiente.


## Árbol de decisión

Este clasificador no ha obtenido muy buenos resultados, pero tampoco se han obtenido resultados pésimos. No es reseñable la inclusión de stop words en este clasificador incluso nos ha bajado un poco la precisión de test. Puede haber sido porque hemos añadido una palabra a la stopList que tenía más información de lo que pensabamos

In [27]:
from sklearn import tree
import numpy as np


# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_preprocessed, train_data.target)

tree_train_predictions = tree_classifier.predict(train_preprocessed)
tree_test_predictions = tree_classifier.predict(test_preprocessed)


print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == train_data.target))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == test_data.target))

Árbol, porcentaje de aciertos en entrenamiento: 0.9999116139296447
Árbol, porcentaje de aciertos en test: 0.554567180031864


Podemos ver el overfit que tiene el árbol porque en el entrenamiento tiene una precisión mucho mas elevada que en la de testing

## Naive Bayes

Obtiene buenos resultados este clasificador. Además, al eliminar las stopwords y el aumento de n-gramas, la precisión mejora notablemente. A mayor sea la stopList mayor será el crecimiento de la precisión.

In [20]:
from sklearn.naive_bayes import MultinomialNB

nb_classifier = MultinomialNB()

nb_classifier.fit(train_preprocessed, train_data.target)

nb_train_predictions = nb_classifier.predict(train_preprocessed)
nb_test_predictions = nb_classifier.predict(test_preprocessed)

print("Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(nb_train_predictions == train_data.target))
print("Naive Bayes, porcentaje de aciertos en test:", np.mean(nb_test_predictions == test_data.target))

Naive Bayes, porcentaje de aciertos en entrenamiento: 0.9673855400388899
Naive Bayes, porcentaje de aciertos en test: 0.7809346787041954


Como podemos observar, al quitar las palabras obtenemos un mayor valor en la precision del test y del train. Cuanto mayor sea la stopList (lista de stopWords) mayor precisión debe sacar el clasificador ya que de esta manera mejora las palabras claves. 

Por otro lado, la eliminación de stopwords puede reducir el recall, lo que hace que sea interesante la indexación del texto completo. 
Por ejemplo, si el usuario está buscando documentos que contienen la frase “ser o no ser”,la eliminación de stopwords hace casi imposible reconocer correctamente los documentos que contienen la frase especificada

## Precision-recall

Hay varias targets que predice muy bien como talk.politics.mideast talk.politics.misc rec.motorcycles pero luego también tenemos talk.politics.guns con una precision muy baja y una recall muy alta

In [26]:
predictions = nb_test_predictions
print(classification_report(test_data.target, predictions, target_names=test_data.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.64      0.71       319
           comp.graphics       0.80      0.65      0.72       389
 comp.os.ms-windows.misc       0.79      0.69      0.74       394
comp.sys.ibm.pc.hardware       0.66      0.77      0.71       392
   comp.sys.mac.hardware       0.87      0.73      0.79       385
          comp.windows.x       0.86      0.78      0.81       395
            misc.forsale       0.88      0.74      0.81       390
               rec.autos       0.83      0.89      0.86       396
         rec.motorcycles       0.93      0.92      0.93       398
      rec.sport.baseball       0.88      0.89      0.89       397
        rec.sport.hockey       0.85      0.97      0.91       399
               sci.crypt       0.64      0.96      0.77       396
         sci.electronics       0.84      0.59      0.69       393
                 sci.med       0.89      0.70      0.78       396
         

Nos ha bajado la precisión muy poco y ha aumentado el recall eso se debe a que nuestra stop list no es la idónea